# Image Captioning with Transformers

In [1]:
!nvidia-smi

Wed Jul  7 00:26:22 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.42.01    Driver Version: 470.42.01    CUDA Version: 11.4     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA GeForce ...  Off  | 00000000:03:00.0 Off |                  N/A |
| N/A   45C    P8    N/A /  N/A |      3MiB /  2004MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
!apt install -qq pigz
%pip install -q timm wandb
%pip install -q --extra-index-url https://developer.download.nvidia.com/compute/redist --upgrade nvidia-dali-cuda110

The following NEW packages will be installed:
  pigz
0 upgraded, 1 newly installed, 0 to remove and 39 not upgraded.
Need to get 57.4 kB of archives.
After this operation, 259 kB of additional disk space will be used.
Selecting previously unselected package pigz.
(Reading database ... 160772 files and directories currently installed.)
Preparing to unpack .../archives/pigz_2.4-1_amd64.deb ...
Unpacking pigz (2.4-1) ...
Setting up pigz (2.4-1) ...
Processing triggers for man-db (2.8.3-2ubuntu0.1) ...
     |████████████████████████████████| 348kB 14.4MB/s 
     |████████████████████████████████| 1.8MB 23.6MB/s 
     |████████████████████████████████| 133kB 65.2MB/s 
     |████████████████████████████████| 102kB 12.5MB/s 
     |████████████████████████████████| 174kB 41.3MB/s 
     |████████████████████████████████| 71kB 9.8MB/s 
     |████████████████████████████████| 613.6MB 26kB/s 


In [ ]:
!git clone https://github.com/ShivamShrirao/Image-Captioning-Transformers

Cloning into 'Image-Captioning-Transformers'...
remote: Enumerating objects: 70, done.
remote: Counting objects: 100% (70/70), done.
remote: Compressing objects: 100% (51/51), done.
remote: Total 70 (delta 29), reused 57 (delta 16), pack-reused 0
Unpacking objects: 100% (70/70), done.


# Download Dataset and Annotations

In [ ]:
!mkdir ~/.kaggle/
!mv kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [ ]:
!kaggle datasets download -d shivamshrirao/coco-trainval2017-320x320

100% 3.46G/3.46G [01:03<00:00, 75.8MB/s]
100% 3.46G/3.46G [01:03<00:00, 58.1MB/s]


In [ ]:
!unzip -q coco-trainval2017-320x320.zip

In [ ]:
# !gdown --id 1-3vdwBlY-CdVultkrFwOhyJTGC5TFUV8

In [ ]:
# !pigz -dc coco_trainval2017_320x320.tar.gz | tar xf -

In [ ]:
from torchvision.datasets.utils import download_and_extract_archive
DATA_DIR = "datasets/COCO"

In [ ]:
download_and_extract_archive("http://images.cocodataset.org/annotations/annotations_trainval2017.zip",
                             download_root=DATA_DIR,
                             remove_finished=True)


Extracting datasets/COCO/annotations_trainval2017.zip to datasets/COCO


In [ ]:
!rm coco-trainval2017-320x320.* datasets/COCO/annotations_trainval2017.zip

In [ ]:
%cd /content/Image-Captioning-Transformers

/content/Image-Captioning-Transformers


In [ ]:
!wandb agent shivamshrirao/Image_Captioning_Transformer/lfj2msgq

# Import libraries

In [ ]:
%cd /content/Image-Captioning-Transformers

/content/Image-Captioning-Transformers


In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
# TODO: Try pre trained CLIP

In [ ]:
import os
import math
import random
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
from tqdm import tqdm

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

In [ ]:
import timm         # torch image models

In [ ]:
plt.rcParams['figure.facecolor'] = 'white'

# Wandb Parameters

In [ ]:
import wandb

In [ ]:
config_defaults = {
    'BATCH_SIZE'        : 256,
    'd_model'           : 512,
    'dim_feedforward'   : 2048,
    'nheads'            : 8,
    'num_decoder_layers': 6,
    'dp_rate'           : 0.2,
    'encoder'           : 'seresnext50_32x4d',
    'activation'        : 'gelu',
    'max_lr'            : 3e-4,
    'betas'             : (0.9, 0.98),
    'eps'               : 1e-9,
    'seed'              : 62134,
    'use_amp'           : True,
    'use_pe'            : True,
    'log_interval'      : 10,
}
CONFIG = config_defaults

In [ ]:
# #hide
# run = wandb.init(id='19sqz0by', project="Image_Captioning_Transformer", resume='must')
# CONFIG = run.config

In [ ]:
run = wandb.init(project="Image_Captioning_Transformer", entity="shivamshrirao", config=config_defaults)
CONFIG = wandb.config

wandb: Currently logged in as: shivamshrirao (use `wandb login --relogin` to force relogin)


In [ ]:
def seed_everything(seed=33):
    np.random.seed(seed)
    random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.benchmark = True
    # torch.backends.cudnn.deterministic = True
    
seed_everything(CONFIG['seed'])

In [ ]:
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

## Read COCO dataset

In [ ]:
from imcap.dataset import *

In [ ]:
DATA_DIR = "../datasets/COCO/"

In [ ]:
train_data = TensorCocoCaptions(root=DATA_DIR+"/train2017/",
                                annFile=DATA_DIR+"/annotations/captions_train2017.json")

val_data = TensorCocoCaptions(root=DATA_DIR+"/val2017/",
                              annFile=DATA_DIR+"/annotations/captions_val2017.json")

loading annotations into memory...
Done (t=1.06s)
creating index...
index created!
loading annotations into memory...
Done (t=0.23s)
creating index...
index created!


## Tokenizer and Build Vocab

In [ ]:
from torchtext.data.utils import get_tokenizer

In [ ]:
tokenizer = get_tokenizer('basic_english')

In [ ]:
def yield_tokens(cap_data):
    for ann in cap_data.coco.anns.values():
        yield tokenizer(ann['caption'])

In [ ]:
special_symbols = ['<unk>', '<pad>', '<bos>', '<eos>']
en_vocab = build_vocab_from_iterator(yield_tokens(train_data), specials=special_symbols, special_first=True)

UNK_IDX, PAD_IDX, BOS_IDX, EOS_IDX = en_vocab(special_symbols)
en_vocab.set_default_index(UNK_IDX)

In [ ]:
len(en_vocab)

28940

In [ ]:
train_data.fill_token_dict(tokenizer, en_vocab, BOS_IDX, EOS_IDX)
val_data.fill_token_dict(tokenizer, en_vocab, BOS_IDX, EOS_IDX)

100%|██████████| 5000/5000 [00:00<00:00, 9689.40it/s]


## Pretrained Glove Embeddings (not used rn)

In [ ]:
# vec.get_vecs_by_tokens(tokens, lower_case_backup=True)

In [ ]:
# vec = torchtext.vocab.GloVe('6B', dim=300)
# unk_vec = vec.vectors.mean(dim=0)
# vec.unk_init = lambda x: unk_vec

# Load dataset into batches

In [ ]:
from imcap.dataloader import *

In [ ]:
nthreads = 2 * len(os.sched_getaffinity(0))
nthreads

8

In [ ]:
train_iter = ExternalInputIterator(train_data, CONFIG['BATCH_SIZE'], PAD_IDX)
pipe = ExternalSourcePipeline(batch_size=CONFIG['BATCH_SIZE'], num_threads=nthreads, device_id=0, external_data=train_iter, input_size=input_size)
train_loader = DALIClassificationIterator(pipe, dynamic_shape=True, auto_reset=True, last_batch_padded=True, size=len(train_iter))

val_iter = ExternalInputIterator(val_data, CONFIG['BATCH_SIZE'], PAD_IDX, training=False)
pipe = ExternalSourcePipeline(batch_size=CONFIG['BATCH_SIZE'], num_threads=nthreads, device_id=0, external_data=val_iter, input_size=input_size, training=False)
val_loader = DALIClassificationIterator(pipe, dynamic_shape=True, auto_reset=True, last_batch_padded=True, size=len(val_iter))

/usr/local/lib/python3.7/dist-packages/nvidia/dali/plugin/base_iterator.py:162: Warning: Please set `reader_name` and don't set last_batch_padded and size manually whenever possible. This may lead, in some situations, to missing some samples or returning duplicated ones. Check the Sharding section of the documentation for more details.
  _iterator_deprecation_warning()


# Initialize Model

In [ ]:
from imcap.layers import *
from imcap.utils import *

In [ ]:
model = CaptionModel(encoder = timm.create_model(CONFIG['encoder'], pretrained=True, num_classes=0, global_pool=''),
                     vocab_size = len(en_vocab),
                     num_decoder_layers = CONFIG['num_decoder_layers'],
                     nheads = CONFIG['nheads'],
                     d_model = CONFIG['d_model'],
                     dim_feedforward = CONFIG['dim_feedforward'],
                     dp_rate = CONFIG['dp_rate'],
                     activation = CONFIG['activation']).to(DEVICE, non_blocking=True)

/usr/local/lib/python3.7/dist-packages/torch/nn/modules/lazy.py:178: UserWarning: Lazy modules are a new feature under heavy development so changes to the API or functionality can happen at any moment.
  warnings.warn('Lazy modules are a new feature under heavy development '


# Learning Rate Schedule

In [ ]:
steps_per_epoch = len(train_loader)

In [ ]:
# def lr_schedule(step, d_model=512, warmup_steps=2*steps_per_epoch):
#     # return 1
#     step = max(1,step)
#     arg1 = step ** -0.5
#     arg2 = step * (warmup_steps ** -1.5)
#     return (d_model ** -0.6) * min(arg1, arg2)

In [ ]:
# plt.plot(list(map(lr_schedule, range(50*steps_per_epoch))))
# plt.show()

In [ ]:
# plt.plot([scheduler.get_last_lr()[0] for _ in range(steps_per_epoch*50) if not scheduler.step()])
# plt.show()

# Loss Function and Optimizer

In [ ]:
loss_fn = nn.CrossEntropyLoss(ignore_index=PAD_IDX)

optimizer = torch.optim.Adam(
    filter(lambda p: p.requires_grad, model.parameters()),
    lr=CONFIG['max_lr'],
    betas=CONFIG['betas'], eps=CONFIG['eps']
)
scheduler = torch.optim.lr_scheduler.OneCycleLR(optimizer, max_lr=CONFIG['max_lr'], total_steps=50*steps_per_epoch, pct_start=0.)
# scheduler = torch.optim.lr_scheduler.LambdaLR(optimizer, lr_schedule)

scaler = torch.cuda.amp.GradScaler(enabled=CONFIG['use_amp'])

In [ ]:
wandb.watch(model, log=None)

# Training functions

In [ ]:
from torch.cuda import amp

In [ ]:
def train_epoch(model, train_loader, optimizer, scaler, scheduler, epoch=1, use_amp=True, log_interval=10):
    model.train()
    model.encoder.eval()
    losses = AverageMeter()
    with tqdm(enumerate(train_loader), total=len(train_loader), desc=f"Epoch {epoch}") as pbar:
        for idx, batch in pbar:
            img, tgt = batch[0]['data'], batch[0]['label'].transpose(0,1)
            # img = img.to(DEVICE, non_blocking=True)
            # tgt = tgt.to(DEVICE, non_blocking=True)
            
            tgt_inp = tgt[:-1,:]      # give input until before the last word.
            tgt_out = tgt[1:, :]      # predict the last word based on input and already predicted sentence. (auto-regressive)

            tgt_mask, tgt_pad_mask = subsequent_mask(tgt_inp.size(0), DEVICE), padding_mask(tgt_inp, PAD_IDX)

            optimizer.zero_grad(set_to_none=True)
            with amp.autocast(enabled=use_amp):
                logits = model(img, tgt_inp, tgt_mask, tgt_pad_mask)
                loss = loss_fn(logits.reshape(-1, logits.size(-1)), tgt_out.reshape(-1))

            scaler.scale(loss).backward()
            scaler.step(optimizer)
            scaler.update()
            scheduler.step()

            losses.update(loss.detach_(), img.size(0))
            del loss, logits, batch, img

            if not idx%log_interval:
                curr_lr = optimizer.param_groups[0]['lr']
                info = {'loss': float(losses.avg), 'lr': curr_lr}
                wandb.log(info)
                pbar.set_postfix(info)

    optimizer.zero_grad(set_to_none=True)
    return float(losses.avg)

In [ ]:
@torch.no_grad()
def evaluate(model, val_loader, use_amp=True):
    model.eval()
    losses = AverageMeter()
    with tqdm(enumerate(val_loader), total=len(val_loader), desc="Evaluating") as pbar:
        for idx, batch in pbar:
            img, tgt = batch[0]['data'], batch[0]['label'].transpose(0,1)
            # img = img.to(DEVICE, non_blocking=True)
            # tgt = tgt.to(DEVICE, non_blocking=True)

            tgt_inp = tgt[:-1,:]      # give input until before the last word.
            tgt_out = tgt[1:, :]      # predict the last word based on input and already predicted sentence. (auto-regressive)

            tgt_mask, tgt_pad_mask = subsequent_mask(tgt_inp.size(0), DEVICE), padding_mask(tgt_inp, PAD_IDX)
            
            with amp.autocast(enabled=use_amp):
                logits = model(img, tgt_inp, tgt_mask, tgt_pad_mask)
                loss = loss_fn(logits.reshape(-1, logits.size(-1)), tgt_out.reshape(-1))

            losses.update(loss.detach_(), img.size(0))
            pbar.set_postfix({'val_loss': float(losses.avg)})
    return float(losses.avg)

# Functions to Make Predictions

In [ ]:
@torch.no_grad()
def greedy_decode(model, img, max_len=100, start_symbol=BOS_IDX):
    model.eval()
    img = img.to(DEVICE, non_blocking=True)
    enc_output = model.encode_image(img)
    tgt = torch.ones(1, 1).fill_(start_symbol).long().to(DEVICE, non_blocking=True)
    for i in range(max_len):
        tgt_mask = subsequent_mask(tgt.size(0), DEVICE)
        out = model.decode_text(tgt, enc_output, tgt_mask)
        out = out.transpose(0,1)
        prob = model.generator(out[:,-1])
        _, next_word = torch.max(prob, dim = 1)
        next_word = next_word.item()
        tgt = torch.cat([tgt, torch.ones(1, 1).fill_(next_word).long().to(DEVICE)], dim=0)
        if next_word == EOS_IDX:
            break
    return tgt.detach()

@torch.no_grad()
def generate_caption(model, img, tgt_vocab):
    tgt = greedy_decode(model, img, max_len=100, start_symbol=BOS_IDX).flatten()
    return " ".join(tgt_vocab.lookup_tokens(tgt.tolist())).replace("<bos>", "").replace("<eos>", "")

# Begin Training

In [ ]:
init_epoch = 1
NUM_EPOCHS = 50

In [ ]:
import gc
gc.collect()
torch.cuda.empty_cache()

In [ ]:
import glob
val_paths = glob.glob(DATA_DIR+"/val2017/*")

In [ ]:
# LR Finder, CLIP, ViT

In [ ]:
#collapse-output
for epoch in range(init_epoch, NUM_EPOCHS+1):
    train_loss = train_epoch(model, train_loader, optimizer, scaler, scheduler,
                             epoch, CONFIG['use_amp'], CONFIG['log_interval'])
    # with torch.no_grad():
    val_loss = evaluate(model, val_loader, CONFIG['use_amp'])

    img = Image.open(random.choice(val_paths))
    caps = generate_caption(model, preproc['val'](img)[None,:], en_vocab)
    wandb.log({"train_loss": train_loss, "val_loss": val_loss, "epoch": epoch, "predictions": wandb.Image(img, caption=caps)})
    print(f"\nEpoch: {epoch}/{NUM_EPOCHS}, Train loss: {train_loss:.3f}, Val loss: {val_loss:.3f}\n")
    gc.collect()
    # if not epoch%10:
    #     save_model(model, optimizer, epoch)

Epoch 1:   0%|          | 0/463 [00:00<?, ?it/s]/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  /pytorch/c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)
Epoch 2:   0%|          | 0/463 [00:00<?, ?it/s]


Epoch: 1/50, Train loss: 3.011, Val loss: 2.646



Epoch 3:   0%|          | 0/463 [00:00<?, ?it/s]


Epoch: 2/50, Train loss: 2.654, Val loss: 2.436



Epoch 4:   0%|          | 0/463 [00:00<?, ?it/s]

...


Epoch: 48/50, Train loss: 2.128, Val loss: 1.940



Epoch 50:   0%|          | 0/463 [00:00<?, ?it/s]


Epoch: 49/50, Train loss: 2.147, Val loss: 1.940



Evaluating: 100%|██████████| 20/20 [00:04<00:00,  4.04it/s, val_loss=1.94]



Epoch: 50/50, Train loss: 2.032, Val loss: 1.940



In [ ]:
init_epoch = epoch
init_epoch

50

In [ ]:
def save_model(model, optimizer, scheduler, epoch=0, path='/content/model.pth'):
    torch.save({
                'projection_head': model.projection_head.state_dict(),
                'decoder': model.decoder.state_dict(),
                'generator': model.generator.state_dict(),
                'optimizer': optimizer.state_dict(),
                'scheduler': scheduler.state_dict(),
                'epoch': epoch,
                }, path)

In [ ]:
def load_model(model, optimizer, scheduler, path='/content/model.pth'):
    checkpoint = torch.load(path, map_location=DEVICE)
    model.projection_head.load_state_dict(checkpoint['projection_head'])
    model.decoder.load_state_dict(checkpoint['decoder'])
    model.generator.load_state_dict(checkpoint['generator'])
    optimizer.load_state_dict(checkpoint['optimizer'])
    scheduler.load_state_dict(checkpoint['scheduler'])

# Make Predictions

In [ ]:
# TODO: Plot attention on images.

In [ ]:
img = Image.open(random.choice(val_paths))
caps = generate_caption(model, preproc['val'](img)[None,:], en_vocab)
# wandb.log({"predictions": wandb.Image(img, caption=caps)})
print(caps)
img

In [ ]:
for i in range(25):
    img = Image.open(random.choice(val_paths))
    caps = generate_caption(model, preproc['val'](img)[None,:], en_vocab)
    wandb.log({"predictions": wandb.Image(img, caption=caps)})

In [ ]:
save_model(model, optimizer, scheduler, epoch=epoch, path=run.dir+'/model.pth')

In [ ]:
run.finish()